In [12]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [170]:
pluto = gpd.read_file('../../rawdata/pluto/manhattan/MNMapPLUTO.shp')

In [171]:
pluto_bk = gpd.read_file('../../rawdata/pluto/brooklyn/BKMapPLUTO.shp')

In [172]:
pluto = pluto.append(pluto_bk)

In [173]:
# translating polygon to lat/lng
crs = {'init': 'epsg:4326'}
pluto = pluto.to_crs(crs)

In [175]:
pluto.tail()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area,geometry
276850,BK,1140,7506,308,203,1004,13,35,11238,E219,...,30407,None,3.011400e+09,2/19/2017,1,17V1,3,321.117183,5335.467431,POLYGON ((-73.96284052535346 40.67870343629913...
276851,BK,958,7503,306,133,2001,15,39,11215,Q001,...,30401,None,3.009580e+09,2/19/2017,1,17V1,3,322.748129,6045.417875,"POLYGON ((-73.98281623938992 40.6761288591569,..."
276852,BK,964,7504,306,133,4000,15,39,11215,E239,...,30402,None,3.009640e+09,8/19/2016,1,17V1,3,379.879612,6857.028356,POLYGON ((-73.98365217495135 40.67509794367613...
276853,BK,2245,7504,301,509,2005,14,33,11206,E230,...,30803,None,3.022450e+09,9/16/2016,1,17V1,3,657.809737,18940.484484,POLYGON ((-73.95089048291159 40.70166773597641...
276854,BK,1153,7504,308,205,2001,13,35,11238,E219,...,30407,None,3.011530e+09,11/26/2016,1,17V1,3,312.524180,5403.064656,POLYGON ((-73.96460783388913 40.67763094274401...


In [176]:
### Consts
datapath = '../../rawdata/sensors/'
filename = datapath + 'nodes.txt'

f = open(filename)
sensors = {}

for line in f:
    print('Calculating for sensor ', s, '...')
    s, lat, lon = line.split(' ')
    
    coord = Point(float(lon), float(lat))
    
    pluto[s] = pluto.apply(lambda row: distanceToPoly(row, coord), axis=1)
    
    minimum = pluto[s].min()

    geom_row = pluto[pluto[s] ==  minimum]
    
    sensors[s] = {}
    sensors[s]['sensor_lat'] = float(lat)
    sensors[s]['sensor_lon'] = float(lon)
    sensors[s]['BBL'] = int(geom_row['BBL'])
#     sensors[s]['BBL_Poly'] = geom_row['geometry']

Calculating for sensor  sonycnode-b827ebf9d204.sonyc ...
Calculating for sensor  sonycnode-b827eb0d8af7.sonyc ...
Calculating for sensor  sonycnode-b827eb0fedda.sonyc ...
Calculating for sensor  sonycnode-b827eb122f0f.sonyc ...
Calculating for sensor  sonycnode-b827eb132382.sonyc ...
Calculating for sensor  sonycnode-b827eb1685c7.sonyc ...
Calculating for sensor  sonycnode-b827eb18a94a.sonyc ...
Calculating for sensor  sonycnode-b827eb252949.sonyc ...
Calculating for sensor  sonycnode-b827eb29eb77.sonyc ...
Calculating for sensor  sonycnode-b827eb2a1bce.sonyc ...
Calculating for sensor  sonycnode-b827eb329ab8.sonyc ...
Calculating for sensor  sonycnode-b827eb32e075.sonyc ...
Calculating for sensor  sonycnode-b827eb3bda47.sonyc ...
Calculating for sensor  sonycnode-b827eb3e842e.sonyc ...
Calculating for sensor  sonycnode-b827eb429cd4.sonyc ...
Calculating for sensor  sonycnode-b827eb42bd4a.sonyc ...
Calculating for sensor  sonycnode-b827eb44506f.sonyc ...
Calculating for sensor  sonycno

In [80]:
def distanceToPoly(row, coord):
    centroid = row['geometry'].centroid
    return centroid.distance(coord)

In [177]:
import json
with open('sensors.json', 'w') as outfile:
    json.dump(sensors, outfile)